## 1. 환경설정

In [1]:
import pandas as pd
from lightgbm import LGBMRegressor

## 2. 데이터 로드

In [2]:
df = pd.read_csv("data.csv", encoding='ansi')
df

,연번,연도,월,주차,공개일,조사기간(S),조사기간(F),긍정,부정,잘모름,...,국제%,북한_외교%,문화%,사회%,사건_사고%,정치%,스포츠%,지역%,"기타(날씨, 미분류)%",퍼센트합
0,1,2012,1,1,2012-01-06,2012-01-02,2012-01-06,26.0,63.0,11,...,9.953095,3.866834,13.980094,13.259352,8.763299,11.943714,6.475232,6.784121,5.880334,100.0
1,2,2012,1,2,2012-01-13,2013-01-09,2012-01-13,24.0,65.0,11,...,11.818475,3.075600,12.216367,13.237983,8.506291,12.345413,6.538337,6.645876,5.871599,100.0
2,3,2012,1,3,2012-01-20,2012-01-16,2012-01-20,22.0,65.0,13,...,12.098789,2.402460,12.118009,13.213531,8.591197,12.742648,6.409764,7.120892,6.515472,100.0
3,4,2012,1,4,2012-01-27,2012-01-25,2012-01-27,24.0,64.0,12,...,13.525856,3.102754,12.397582,13.472129,8.623237,10.100739,6.662189,6.675621,6.836803,100.0
4,5,2012,2,1,NaN,2012-01-30,2012-02-03,22.0,65.0,13,...,10.338562,2.247513,13.274675,13.771997,8.645754,11.027161,7.067712,7.096404,6.685157,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,607,2023,8,1,2023-08-04,2023-08-01,2023-08-03,33.0,56.0,11,...,10.009558,2.710922,7.863411,18.463811,8.523764,10.278912,4.674602,13.293944,7.846034,NaN
607,608,2023,8,2,2023-08-11,2023-08-08,2023-08-10,35.0,57.0,8,...,10.478589,1.410579,7.867338,15.633921,12.216625,8.413098,3.912678,13.853904,12.418136,NaN
608,609,2023,8,3,미실시,2023-08-15,2023-08-17,NaN,NaN,100,...,12.550154,3.965662,8.967062,14.817580,9.452272,10.711953,4.404218,13.473920,6.662312,NaN
609,610,2023,8,4,2023-08-25,2023-08-22,2023-08-24,34.0,57.0,9,...,13.380852,6.415033,7.297910,13.672445,11.501701,9.881743,4.122793,12.214482,5.451158,NaN


In [3]:
df.columns

Index(['연번', '연도', '월', '주차', '공개일', '조사기간(S)', '조사기간(F)', '긍정', '부정', '잘모름',
       '과학_IT', '경제', '국제', '북한_외교', '문화', '사회', '사건_사고', '정치', '스포츠', '지역',
       '기타(날씨, 미분류)', '주간합', '과학_IT%', '경제%', '국제%', '북한_외교%', '문화%', '사회%',
       '사건_사고%', '정치%', '스포츠%', '지역%', '기타(날씨, 미분류)%', '퍼센트합'],
      dtype='object')

### 2.1. 전처리

In [4]:
# 필요한 열만 추출

df_train = df.loc[:,['연도', '월', '주차', '긍정', '부정', '잘모름',
       '과학_IT', '경제', '국제', '북한_외교', '문화', '사회', '사건_사고', '정치', '스포츠', '지역',
       '기타(날씨, 미분류)', '주간합', '과학_IT%', '경제%', '국제%', '북한_외교%', '문화%', '사회%',
       '사건_사고%', '정치%', '스포츠%', '지역%', '기타(날씨, 미분류)%']]

df_train

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,...,경제%,국제%,북한_외교%,문화%,사회%,사건_사고%,정치%,스포츠%,지역%,"기타(날씨, 미분류)%"
0,2012,1,1,26.0,63.0,11,334,1335,870,338,...,15.272852,9.953095,3.866834,13.980094,13.259352,8.763299,11.943714,6.475232,6.784121,5.880334
1,2012,1,2,24.0,65.0,11,372,1464,1099,286,...,15.743628,11.818475,3.075600,12.216367,13.237983,8.506291,12.345413,6.538337,6.645876,5.871599
2,2012,1,3,22.0,65.0,13,455,1500,1259,250,...,14.414761,12.098789,2.402460,12.118009,13.213531,8.591197,12.742648,6.409764,7.120892,6.515472
3,2012,1,4,24.0,64.0,12,319,1066,1007,231,...,14.318334,13.525856,3.102754,12.397582,13.472129,8.623237,10.100739,6.662189,6.675621,6.836803
4,2012,2,1,22.0,65.0,13,447,1628,1081,235,...,15.570008,10.338562,2.247513,13.274675,13.771997,8.645754,11.027161,7.067712,7.096404,6.685157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2023,8,1,33.0,56.0,11,418,1462,1152,312,...,12.703102,10.009558,2.710922,7.863411,18.463811,8.523764,10.278912,4.674602,13.293944,7.846034
607,2023,8,2,35.0,57.0,8,399,1244,1248,168,...,10.445004,10.478589,1.410579,7.867338,15.633921,12.216625,8.413098,3.912678,13.853904,12.418136
608,2023,8,3,NaN,NaN,100,362,1245,1345,425,...,11.617057,12.550154,3.965662,8.967062,14.817580,9.452272,10.711953,4.404218,13.473920,6.662312
609,2023,8,4,34.0,57.0,9,486,1497,1652,792,...,12.125385,13.380852,6.415033,7.297910,13.672445,11.501701,9.881743,4.122793,12.214482,5.451158


In [5]:
# 데이터 셋 체크

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연도            611 non-null    int64  
 1   월             611 non-null    int64  
 2   주차            611 non-null    int64  
 3   긍정            538 non-null    float64
 4   부정            538 non-null    float64
 5   잘모름           611 non-null    int64  
 6   과학_IT         611 non-null    int64  
 7   경제            611 non-null    int64  
 8   국제            611 non-null    int64  
 9   북한_외교         611 non-null    int64  
 10  문화            611 non-null    int64  
 11  사회            611 non-null    int64  
 12  사건_사고         611 non-null    int64  
 13  정치            611 non-null    int64  
 14  스포츠           611 non-null    int64  
 15  지역            611 non-null    int64  
 16  기타(날씨, 미분류)   611 non-null    int64  
 17  주간합           611 non-null    int64  
 18  과학_IT%        611 non-null    

약 73주치 데이터 손실

전체 데이터 611건 중 73주치 데이터 손실

538건 데이터 활용 가능

In [6]:
# null 값 채우기
df_clean = df_train.fillna(0)
df_clean.describe()

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,...,경제%,국제%,북한_외교%,문화%,사회%,사건_사고%,정치%,스포츠%,지역%,"기타(날씨, 미분류)%"
count,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,...,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000,611.000000
mean,2017.340426,6.445172,2.708674,38.346972,40.088380,21.564648,447.004910,1424.198036,1201.700491,471.726678,...,13.332254,11.315574,4.557031,12.932907,13.533115,8.078357,12.893038,5.921987,8.076076,5.178689
std,3.374735,3.437239,1.275330,19.839373,20.345025,29.215710,119.105567,329.692835,300.657226,273.195124,...,1.800079,2.087071,2.672817,4.108673,2.656315,1.647299,4.000452,2.352242,2.745493,1.060880
min,2012.000000,1.000000,1.000000,0.000000,0.000000,3.000000,144.000000,459.000000,316.000000,105.000000,...,7.534083,7.729514,1.212983,6.465931,8.023142,2.840909,6.022727,3.094855,3.303303,3.061679
25%,2014.000000,3.000000,2.000000,29.000000,26.500000,8.000000,358.000000,1161.000000,1006.000000,290.000000,...,12.104833,9.928503,2.733055,9.669703,11.694202,6.994063,10.539793,4.422173,6.110021,4.430945
50%,2017.000000,6.000000,3.000000,40.000000,46.000000,10.000000,445.000000,1446.000000,1152.000000,399.000000,...,13.337530,11.045752,3.893724,12.096019,13.019488,7.942318,12.131716,5.172414,6.957756,5.040414
75%,2020.000000,9.000000,4.000000,48.000000,55.000000,14.000000,525.000000,1672.500000,1356.000000,585.000000,...,14.465869,12.252532,5.553908,15.016035,14.708250,9.059974,14.165800,7.070410,10.607597,5.730082
max,2023.000000,12.000000,5.000000,84.000000,93.000000,100.000000,913.000000,2402.000000,2958.000000,2896.000000,...,18.822496,24.070993,22.282130,28.566052,26.233806,13.448985,37.658455,22.753308,20.788838,12.418136


In [7]:
## 확인
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연도            611 non-null    int64  
 1   월             611 non-null    int64  
 2   주차            611 non-null    int64  
 3   긍정            611 non-null    float64
 4   부정            611 non-null    float64
 5   잘모름           611 non-null    int64  
 6   과학_IT         611 non-null    int64  
 7   경제            611 non-null    int64  
 8   국제            611 non-null    int64  
 9   북한_외교         611 non-null    int64  
 10  문화            611 non-null    int64  
 11  사회            611 non-null    int64  
 12  사건_사고         611 non-null    int64  
 13  정치            611 non-null    int64  
 14  스포츠           611 non-null    int64  
 15  지역            611 non-null    int64  
 16  기타(날씨, 미분류)   611 non-null    int64  
 17  주간합           611 non-null    int64  
 18  과학_IT%        611 non-null    

In [8]:
## 학습용 데이터 셋 저장

df_clean.to_csv("data_clean.csv",encoding = 'cp949',index = False)

## 3. 모델링 진행

In [9]:
df = pd.read_csv("data_clean.csv",encoding = 'cp949')
df

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,...,경제%,국제%,북한_외교%,문화%,사회%,사건_사고%,정치%,스포츠%,지역%,"기타(날씨, 미분류)%"
0,2012,1,1,26.0,63.0,11,334,1335,870,338,...,15.272852,9.953095,3.866834,13.980094,13.259352,8.763299,11.943714,6.475232,6.784121,5.880334
1,2012,1,2,24.0,65.0,11,372,1464,1099,286,...,15.743628,11.818475,3.075600,12.216367,13.237983,8.506291,12.345413,6.538337,6.645876,5.871599
2,2012,1,3,22.0,65.0,13,455,1500,1259,250,...,14.414761,12.098789,2.402460,12.118009,13.213531,8.591197,12.742648,6.409764,7.120892,6.515472
3,2012,1,4,24.0,64.0,12,319,1066,1007,231,...,14.318334,13.525856,3.102754,12.397582,13.472129,8.623237,10.100739,6.662189,6.675621,6.836803
4,2012,2,1,22.0,65.0,13,447,1628,1081,235,...,15.570008,10.338562,2.247513,13.274675,13.771997,8.645754,11.027161,7.067712,7.096404,6.685157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2023,8,1,33.0,56.0,11,418,1462,1152,312,...,12.703102,10.009558,2.710922,7.863411,18.463811,8.523764,10.278912,4.674602,13.293944,7.846034
607,2023,8,2,35.0,57.0,8,399,1244,1248,168,...,10.445004,10.478589,1.410579,7.867338,15.633921,12.216625,8.413098,3.912678,13.853904,12.418136
608,2023,8,3,0.0,0.0,100,362,1245,1345,425,...,11.617057,12.550154,3.965662,8.967062,14.817580,9.452272,10.711953,4.404218,13.473920,6.662312
609,2023,8,4,34.0,57.0,9,486,1497,1652,792,...,12.125385,13.380852,6.415033,7.297910,13.672445,11.501701,9.881743,4.122793,12.214482,5.451158


In [10]:
## 미조사 데이터 제외
df_clean = df.loc[df['긍정'] != 0]
df_clean = df_clean.reset_index(drop = True)
df_clean

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,...,경제%,국제%,북한_외교%,문화%,사회%,사건_사고%,정치%,스포츠%,지역%,"기타(날씨, 미분류)%"
0,2012,1,1,26.0,63.0,11,334,1335,870,338,...,15.272852,9.953095,3.866834,13.980094,13.259352,8.763299,11.943714,6.475232,6.784121,5.880334
1,2012,1,2,24.0,65.0,11,372,1464,1099,286,...,15.743628,11.818475,3.075600,12.216367,13.237983,8.506291,12.345413,6.538337,6.645876,5.871599
2,2012,1,3,22.0,65.0,13,455,1500,1259,250,...,14.414761,12.098789,2.402460,12.118009,13.213531,8.591197,12.742648,6.409764,7.120892,6.515472
3,2012,1,4,24.0,64.0,12,319,1066,1007,231,...,14.318334,13.525856,3.102754,12.397582,13.472129,8.623237,10.100739,6.662189,6.675621,6.836803
4,2012,2,1,22.0,65.0,13,447,1628,1081,235,...,15.570008,10.338562,2.247513,13.274675,13.771997,8.645754,11.027161,7.067712,7.096404,6.685157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,2023,7,4,35.0,55.0,10,474,1591,1052,465,...,12.859683,8.503071,3.758487,7.363401,17.119302,10.539929,10.483349,4.704171,14.354995,6.482380
534,2023,8,1,33.0,56.0,11,418,1462,1152,312,...,12.703102,10.009558,2.710922,7.863411,18.463811,8.523764,10.278912,4.674602,13.293944,7.846034
535,2023,8,2,35.0,57.0,8,399,1244,1248,168,...,10.445004,10.478589,1.410579,7.867338,15.633921,12.216625,8.413098,3.912678,13.853904,12.418136
536,2023,8,4,34.0,57.0,9,486,1497,1652,792,...,12.125385,13.380852,6.415033,7.297910,13.672445,11.501701,9.881743,4.122793,12.214482,5.451158


In [11]:
df_clean.columns

Index(['연도', '월', '주차', '긍정', '부정', '잘모름', '과학_IT', '경제', '국제', '북한_외교', '문화',
       '사회', '사건_사고', '정치', '스포츠', '지역', '기타(날씨, 미분류)', '주간합', '과학_IT%', '경제%',
       '국제%', '북한_외교%', '문화%', '사회%', '사건_사고%', '정치%', '스포츠%', '지역%',
       '기타(날씨, 미분류)%'],
      dtype='object')

In [12]:
len(df_clean)

538

In [23]:
## 전월 긍정_부정_잘모름 열 추가
for i in range(1,len(df_clean)):
    df_clean.loc[i,'긍정_과거'] = df_clean.loc[i-1,'긍정']
    df_clean.loc[i,'부정_과거'] = df_clean.loc[i-1,'부정']
    df_clean.loc[i,'잘모름_과거'] = df_clean.loc[i-1,'잘모름']
    
    ## 뉴스 카테고리별 증감량 열 추가
    df_clean.loc[i,'과학_IT_증감량'] = df_clean.loc[i,'과학_IT'] - df_clean.loc[i-1,'과학_IT']
    df_clean.loc[i,'경제_증감량'] = df_clean.loc[i,'경제'] - df_clean.loc[i-1,'경제']
    df_clean.loc[i,'국제_증감량'] = df_clean.loc[i,'국제'] - df_clean.loc[i-1,'국제']
    df_clean.loc[i,'북한_외교_증감량'] = df_clean.loc[i,'북한_외교'] - df_clean.loc[i-1,'북한_외교']
    df_clean.loc[i,'문화_증감량'] = df_clean.loc[i,'문화'] - df_clean.loc[i-1,'문화']
    df_clean.loc[i,'사회_증감량'] = df_clean.loc[i,'사회'] - df_clean.loc[i-1,'사회']
    df_clean.loc[i,'사건_사고_증감량'] = df_clean.loc[i,'사건_사고'] - df_clean.loc[i-1,'사건_사고']
    df_clean.loc[i,'정치_증감량'] = df_clean.loc[i,'정치'] - df_clean.loc[i-1,'정치']
    df_clean.loc[i,'스포츠_증감량'] = df_clean.loc[i,'스포츠'] - df_clean.loc[i-1,'스포츠']
    df_clean.loc[i,'지역_증감량'] = df_clean.loc[i,'지역'] - df_clean.loc[i-1,'지역']
    df_clean.loc[i,'기타(날씨, 미분류)_증감량'] = df_clean.loc[i,'기타(날씨, 미분류)'] - df_clean.loc[i-1,'기타(날씨, 미분류)']
    
    
    
df_clean = df_clean.drop(0,axis=0).reset_index(drop = True)

In [25]:
display(df_clean)

display(df_clean.columns)

,연도,월,주차,긍정,부정,잘모름,과학_IT,경제,국제,북한_외교,...,경제_증감량,국제_증감량,북한_외교_증감량,문화_증감량,사회_증감량,사건_사고_증감량,정치_증감량,스포츠_증감량,지역_증감량,"기타(날씨, 미분류)_증감량"
0,2012,1,3,22.0,65.0,13,455,1500,1259,250,...,36.0,160.0,-36.0,125.0,144.0,103.0,178.0,59.0,123.0,132.0
1,2012,1,4,24.0,64.0,12,319,1066,1007,231,...,-434.0,-252.0,-19.0,-338.0,-372.0,-252.0,-574.0,-171.0,-244.0,-169.0
2,2012,2,1,22.0,65.0,13,447,1628,1081,235,...,562.0,74.0,4.0,465.0,437.0,262.0,401.0,243.0,245.0,190.0
3,2012,2,2,24.0,64.0,12,382,1632,1239,260,...,4.0,158.0,25.0,-66.0,-15.0,-67.0,160.0,67.0,-5.0,-98.0
4,2012,2,3,23.0,62.0,15,364,1485,1263,305,...,-147.0,24.0,45.0,111.0,-163.0,139.0,-104.0,63.0,-155.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,2023,7,4,35.0,55.0,10,474,1591,1052,465,...,195.0,-141.0,-81.0,29.0,-156.0,417.0,62.0,130.0,-891.0,-43.0
532,2023,8,1,33.0,56.0,11,418,1462,1152,312,...,-129.0,100.0,-153.0,-6.0,7.0,-323.0,-114.0,-44.0,-246.0,101.0
533,2023,8,2,35.0,57.0,8,399,1244,1248,168,...,-218.0,96.0,-144.0,32.0,-263.0,474.0,-181.0,-72.0,120.0,576.0
534,2023,8,4,34.0,57.0,9,486,1497,1652,792,...,253.0,404.0,624.0,-36.0,-174.0,-35.0,218.0,43.0,-142.0,-806.0


Index(['연도', '월', '주차', '긍정', '부정', '잘모름', '과학_IT', '경제', '국제', '북한_외교', '문화',
       '사회', '사건_사고', '정치', '스포츠', '지역', '기타(날씨, 미분류)', '주간합', '과학_IT%', '경제%',
       '국제%', '북한_외교%', '문화%', '사회%', '사건_사고%', '정치%', '스포츠%', '지역%',
       '기타(날씨, 미분류)%', '긍정_과거', '부정_과거', '잘모름_과거', '과학_IT_증감량', '경제_증감량',
       '국제_증감량', '북한_외교_증감량', '문화_증감량', '사회_증감량', '사건_사고_증감량', '정치_증감량',
       '스포츠_증감량', '지역_증감량', '기타(날씨, 미분류)_증감량'],
      dtype='object')

In [26]:
# 학습용 데이터
X_train = df_clean.loc[df_clean['연도'] < 2023,['긍정_과거', '부정_과거', '잘모름_과거', '과학_IT', '경제', '국제', '북한_외교', '문화',
       '사회', '사건_사고', '정치', '스포츠', '지역', '기타(날씨, 미분류)', '주간합', '과학_IT%', '경제%',
       '국제%', '북한_외교%', '문화%', '사회%', '사건_사고%', '정치%', '스포츠%', '지역%', '기타(날씨, 미분류)%',
        '과학_IT_증감량', '경제_증감량', '국제_증감량', '북한_외교_증감량', '문화_증감량', '사회_증감량', '사건_사고_증감량', '정치_증감량',
       '스포츠_증감량', '지역_증감량', '기타(날씨, 미분류)_증감량']]
y_train = df_clean.loc[df_clean['연도'] < 2023,['긍정']]

# y_train = df_clean.loc[df_clean['연도'] < 2022,['부정']]

# 테스트용 데이터
X_test = df_clean.loc[df_clean['연도'] == 2023,['긍정_과거', '부정_과거', '잘모름_과거','과학_IT', '경제', '국제', '북한_외교', '문화',
       '사회', '사건_사고', '정치', '스포츠', '지역', '기타(날씨, 미분류)', '주간합', '과학_IT%', '경제%',
       '국제%', '북한_외교%', '문화%', '사회%', '사건_사고%', '정치%', '스포츠%', '지역%', '기타(날씨, 미분류)%',
         '과학_IT_증감량', '경제_증감량', '국제_증감량', '북한_외교_증감량', '문화_증감량', '사회_증감량', '사건_사고_증감량', '정치_증감량',
       '스포츠_증감량', '지역_증감량', '기타(날씨, 미분류)_증감량']]
y_test = df_clean.loc[df_clean['연도'] == 2023,['긍정']]

# y_test = df_clean.loc[df_clean['연도'] == 2022,['부정']]

y_idx = df_clean.loc[df_clean['연도'] == 2023,['연도', '월', '주차']]

In [27]:
y_test

,긍정
504,37.0
505,35.0
506,36.0
507,34.0
508,32.0
509,35.0
510,37.0
511,36.0
512,34.0
513,33.0


In [28]:
X_train.columns = ["Positive_Past","Negative_Past","Non_Past","Science_IT","Ecnomic","Global","North_Korea","Culture","Social","Issue", 'Politic', 'Sports', 'Local', 'Etc', 'SUM',
                  "Science_IT_P","Ecnomic_P","Global_P","North_Korea_P","Culture_P","Social_P","Issue_P", 'Politic_P', 'Sports_P', 'Local_P', 'Etc_P',
                  "Science_IT_ID","Ecnomic_ID","Global_ID","North_Korea_ID","Culture_ID","Social_ID","Issue_ID", 'Politic_ID', 'Sports_ID', 'Local_ID', 'Etc_ID']
y_train.columns = ['Target']

X_test.columns = ["Positive_Past","Negative_Past","Non_Past","Science_IT","Ecnomic","Global","North_Korea","Culture","Social","Issue", 'Politic', 'Sports', 'Local', 'Etc', 'SUM',
                  "Science_IT_P","Ecnomic_P","Global_P","North_Korea_P","Culture_P","Social_P","Issue_P", 'Politic_P', 'Sports_P', 'Local_P', 'Etc_P',
                  "Science_IT_ID","Ecnomic_ID","Global_ID","North_Korea_ID","Culture_ID","Social_ID","Issue_ID", 'Politic_ID', 'Sports_ID', 'Local_ID', 'Etc_ID']
y_test.columns = ['Target']

In [29]:
## 모델링

lgbm = LGBMRegressor()
lgbm.fit(X_train,y_train)

lgbm.score(X_train,y_train)

0.9919213160130665

In [30]:
y_pred = lgbm.predict(X_test)

In [31]:
# 결과 해석

result = pd.DataFrame([y_pred, y_test.Target]).T
result.columns = ['예측',"실제값"]
result = pd.concat([y_idx.reset_index(drop=True),result.reset_index(drop=True)],axis = 1)
result

,연도,월,주차,예측,실제값
0,2023,1,1,38.185821,37.0
1,2023,1,2,35.942030,35.0
2,2023,1,3,35.492412,36.0
3,2023,2,1,37.077018,34.0
4,2023,2,2,36.773600,32.0
5,2023,2,3,32.269157,35.0
6,2023,2,4,37.062892,37.0
7,2023,3,1,37.070276,36.0
8,2023,3,2,34.772727,34.0
9,2023,3,3,34.467469,33.0


In [32]:
# 변수 중요도
feature_importance = pd.DataFrame([lgbm.feature_name_, lgbm.feature_importances_]).T
feature_importance.sort_values(by = 1, ascending = False)

,0,1
0,Positive_Past,177
22,Politic_P,92
5,Global,78
10,Politic,73
12,Local,67
15,Science_IT_P,64
32,Issue_ID,62
25,Etc_P,60
16,Ecnomic_P,58
13,Etc,57


In [33]:
## 성능 지표 확인
from sklearn.metrics import mean_absolute_error

metric_score = mean_absolute_error(y_pred, y_test.Target)
display(metric_score)

2.2500202599125068